In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("d:data/startup/master_readmission_dataset.csv")
df.head()

C:\Users\syeds\AppData\Local\Temp\ipykernel_22288\2760171154.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("d:data/startup/master_readmission_dataset.csv")


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,readmit_30,discharge_disposition,admission_type,admission_source
0,2278392,8222157,Caucasian,Female,[0-10),NaN,6,25,1,1,...,No,No,No,No,No,NO,0,Not Mapped,Unknown,Physician Referral
1,149190,55629189,Caucasian,Female,[10-20),NaN,1,1,7,3,...,No,No,No,Ch,Yes,>30,0,Discharged to home,Emergency,Emergency Room
2,64410,86047875,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,...,No,No,No,No,Yes,NO,0,Discharged to home,Emergency,Emergency Room
3,500364,82442376,Caucasian,Male,[30-40),NaN,1,1,7,2,...,No,No,No,Ch,Yes,NO,0,Discharged to home,Emergency,Emergency Room
4,16680,42519267,Caucasian,Male,[40-50),NaN,1,1,7,1,...,No,No,No,Ch,Yes,NO,0,Discharged to home,Emergency,Emergency Room


In [3]:
df.columns

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted',
       'readmit_30', 'discharge_disposition', 'admission_type',
       'admission_source'],


In [4]:
#drop expired from discharge disposition
if "discharge_dispostion" in df.columns:
    before = len(df)
    df =df[df["discharge_disposition"] != "Expired"].copy()
    print("Dropped Expired", before - len(df))

In [6]:
#Replacing ? if any
df = df.replace("?", np.nan)

# Feature Engineering

In [12]:
# 1) LOS(length of stay) bucket
df["los_bucket"] = pd.cut(
    df["time_in_hospital"],
    bins=[0, 2, 5, 10, 100],
    labels=["0-2", "3-5", "6-10", "10+"],
    include_lowest=True
)

# 2) Prior utilization total
df["prior_util_total"] = (
    df["number_inpatient"] +
    df["number_emergency"] +
    df["number_outpatient"]
)

# 3) Clean medication buckets (no of meds)
df["meds_bucket_clean"] = pd.cut(
    df["num_medications"],
    bins=[0, 9, 13, 17, 22, 100],
    labels=["1-9", "10-13", "14-17", "18-22", "23+"]
)

# 4) High meds flag (top quartile)
q75_meds = df["num_medications"].quantile(0.75)
df["high_meds_flag"] = (df["num_medications"] >= q75_meds).astype(int)

print("q75 meds:", q75_meds)


q75 meds: 20.0


In [13]:
drop_cols = [
    "encounter_id", "patient_nbr",  # identifiers
    "weight",                       # missing/noisy
    "diag_1", "diag_2", "diag_3"    # ICD codes need grouping
]

df_model = df.drop(columns=[c for c in drop_cols if c in df.columns], errors="ignore").copy()

print("Model-ready shape:", df_model.shape)
print("Columns added:", ["los_bucket","prior_util_total","meds_bucket_clean","high_meds_flag"])


Model-ready shape: (101766, 52)
Columns added: ['los_bucket', 'prior_util_total', 'meds_bucket_clean', 'high_meds_flag']


In [17]:
df.head(20
       )

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,diabetesMed,readmitted,readmit_30,discharge_disposition,admission_type,admission_source,los_bucket,prior_util_total,meds_bucket_clean,high_meds_flag
0,2278392,8222157,Caucasian,Female,[0-10),NaN,6,25,1,1,...,No,NO,0,Not Mapped,Unknown,Physician Referral,0-2,0,1-9,0
1,149190,55629189,Caucasian,Female,[10-20),NaN,1,1,7,3,...,Yes,>30,0,Discharged to home,Emergency,Emergency Room,3-5,0,18-22,0
2,64410,86047875,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,...,Yes,NO,0,Discharged to home,Emergency,Emergency Room,0-2,3,10-13,0
3,500364,82442376,Caucasian,Male,[30-40),NaN,1,1,7,2,...,Yes,NO,0,Discharged to home,Emergency,Emergency Room,0-2,0,14-17,0
4,16680,42519267,Caucasian,Male,[40-50),NaN,1,1,7,1,...,Yes,NO,0,Discharged to home,Emergency,Emergency Room,0-2,0,1-9,0
5,35754,82637451,Caucasian,Male,[50-60),NaN,2,1,2,3,...,Yes,>30,0,Discharged to home,Urgent,Clinic Referral,3-5,0,14-17,0
6,55842,84259809,Caucasian,Male,[60-70),NaN,3,1,2,4,...,Yes,NO,0,Discharged to home,Elective,Clinic Referral,3-5,0,18-22,1
7,63768,114882984,Caucasian,Male,[70-80),NaN,1,1,7,5,...,Yes,>30,0,Discharged to home,Emergency,Emergency Room,3-5,0,10-13,0
8,12522,48330783,Caucasian,Female,[80-90),NaN,2,1,4,13,...,Yes,NO,0,Discharged to home,Urgent,Transfer from a hospital,10+,0,23+,1
9,15738,63555939,Caucasian,Female,[90-100),NaN,3,3,4,12,...,Yes,NO,0,Discharged/transferred to SNF,Elective,Transfer from a hospital,10+,0,18-22,0


In [18]:
# model data
OUT = r"D:\data\startup\hospital_readmission_model_ready.csv"
df_model.to_csv(OUT, index=False)
print("Saved:", OUT)


Saved: D:\data\startup\hospital_readmission_model_ready.csv
